In [1]:
import datetime
import Image
import gc
import numpy as np
import os
import random
from scipy import misc
import string
import time

# Set some Theano config before initializing
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cpu,floatX=float32,allow_gc=False,openmp=True"
import theano

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

import emotion_model
import dwdii_transforms

random.seed(20275)

Using Theano backend.


In [2]:
print "device:", theano.config.device
print "floatX:",  theano.config.floatX
print "mode:", theano.config.mode
print "openmp:", theano.config.openmp
print "allow_gc:", theano.config.allow_gc

device: cpu
floatX: float32
mode: FAST_RUN
openmp: True
allow_gc: False


In [3]:
imagePath = "/root/facial_expressions/images"
dataPath = "/root/facial_expressions/data/legend.csv"
imgResize = (150, 150)

In [4]:
os.listdir('/root/facial_expressions/data')

['legend.csv', '500_picts_satz.csv']

### Load Training and Test Data

In [5]:
#emoMetaData = dwdii_transforms.load_training_metadata(dataPath, True)

In [6]:
#len(emoMetaData)

In [7]:
maxData = 1820
X_data, Y_data = dwdii_transforms.load_data(dataPath, imagePath, maxData = maxData, verboseFreq = 200, imgResize=imgResize)
print X_data.shape
print Y_data.shape

0.000000: King_Abdullah_II_0001.jpg
0.109890: Lleyton_Hewitt_0041.jpg
0.219780: Dwarakish_30.jpg
0.219780: Megawati_Sukarnoputri_0018.jpg
0.329670: HrithikRoshan_174.jpg
0.329670: Valerie_Thwaites_0001.jpg
0.439560: HrithikRoshan_115.jpg
0.439560: Dwarakish_180.jpg
0.439560: HrithikRoshan_3.jpg
0.439560: George_W_Bush_0422.jpg
0.549451: Serena_Williams_0015.jpg
0.659341: Bob_Riley_0001.jpg
0.769231: Lindsay_Davenport_0012.jpg
0.879121: HrithikRoshan_112.jpg
0.879121: Sergio_Vieira_De_Mello_0002.jpg
0.989011: Trent_Lott_0016.jpg
(1817, 150, 150)
(1817, 1)


### Transformations
In this section, we will apply transformations to the existing images to increase of training data, as well as add a bit of noise in the hopes of improving the overall training activities.

In [8]:
newImgs = np.zeros([X_data.shape[0] * 2, X_data.shape[1], X_data.shape[2]])
newYs = np.zeros([Y_data.shape[0] * 2, Y_data.shape[1]], dtype=np.int8)
print newImgs.shape
print newYs.shape

(3634, 150, 150)
(3634, 1)


In [ ]:
ndx = 0
for i in range(X_data.shape[0]):
    img = X_data[i]
    #print img[:]
    #im = Image.fromarray(img)
    #print img.shape
    #s = list(img.shape)
    #print len(s)
    
    #fig = plt.figure(figsize=(350,350))
    #plt.imshow(img)
    #plt.axis('off')
    #plt.tight_layout()  
    
    #fig.savefig("test.png", dpi=fig.dpi)
    #plt.close(fig)
    
    #img *= 255.0
    #img = img.astype("int8")
    
    
    img1 = dwdii_transforms.reflectY(img)
    newImgs[ndx] = img1
    newYs[ndx] = Y_data[i]
    misc.imsave("test1.png", img1)
    ndx += 1
    
    img2 = dwdii_transforms.rotate5(img)
    newImgs[ndx] = img2
    newYs[ndx] = Y_data[i]
    misc.imsave("test2.png", img2)
    ndx += 1

    #break
    
print("Done", str(datetime.datetime.now()))

In [ ]:
gc.collect()

In [ ]:
X_data2 = np.concatenate((X_data, newImgs))
Y_data2 = np.concatenate((Y_data, newYs))
print X_data2.shape
print Y_data2.shape

### Split Training/Test Sets

The following code segment splits the data into training and test data sets. Currently this is just a standard 80/20 split for training and test respectively. A random.shuffle should be probably be added here as well.

In [9]:
skippedTransforms = True
if skippedTransforms:
    X_data2 = X_data
    Y_data2 = Y_data

In [10]:
# Split the data into Training and Test sets
trainNdx = int(X_data2.shape[0] * .8)
print trainNdx
X_train, X_test = np.split(X_data2, [trainNdx])
Y_train, Y_test = np.split(Y_data2, [trainNdx])
print X_train.shape
print X_test.shape

print Y_train.shape
print Y_test.shape

1453
(1453, 150, 150)
(364, 150, 150)
(1453, 1)
(364, 1)


### Define the Model
In this section, we define the model. The `emotion_model` module contains the model definition itself. `emotion_model_v1` is a basic convolutional neural network.

In [11]:
# Map the emotions to integers for categorization later.
emotions = dwdii_transforms.emotionNumerics()
print emotions
print len(emotions)

{'sadness': 6, 'neutral': 2, 'contempt': 7, 'disgust': 1, 'anger': 0, 'surprise': 4, 'fear': 5, 'happiness': 3}
8


In [13]:
#model = emotion_model.emotion_model_v3_2(len(emotions), verbose=True)
model = emotion_model.cnn_model_jhamski(len(emotions), verbose=True, 
                                        input_shape=(1,X_train.shape[1],X_train.shape[2]))
#print(model.summary())

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
convolution2d_1 (Convolution2D)    (None, 32, 148, 148)320         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)          (None, 32, 148, 148)0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)      (None, 32, 74, 74)  0           activation_1[0][0]               
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)    (None, 32, 72, 72)  9248        maxpooling2d_1[0][0]             
___________________________________________________________________________________________

### Training the Model

The following code segment trains the model using the `run_network` helper function. I seem to be hitting a memory issue (my interpretation), when I have batches above a certain threshold. Batches=10 work fine, but batches of 100 are too big. May need to allocate more RAM to the docker container.

In [15]:
# Reshape to the appropriate shape for the CNN input
testX = X_test.reshape(X_test.shape[0], 1, X_train.shape[1],X_train.shape[2])
trainX = X_train.reshape(X_train.shape[0], 1, X_train.shape[1],X_train.shape[2])

In [16]:
loadWeights = False
if loadWeights:
    model.load_weights("dwdii-emo-01vjh-1-Cloud.hdf5")

In [23]:
print "Training start: " + str(datetime.datetime.now())
m, h = emotion_model.run_network([trainX, testX, Y_train, Y_test], model, batch=300, epochs=10, verbosity=1)

Training start: 2016-11-28 01:59:11.696304
(1453, 8)
(364, 8)
Training model...
Train on 1453 samples, validate on 364 samples
Epoch 1/10
1453/1453 [==============================] - 55s - loss: 0.0137 - acc: 0.9807 - val_loss: 0.5012 - val_acc: 0.6484
Epoch 2/10
1453/1453 [==============================] - 55s - loss: 0.0170 - acc: 0.9732 - val_loss: 0.4920 - val_acc: 0.6593
Epoch 3/10
1453/1453 [==============================] - 55s - loss: 0.0131 - acc: 0.9780 - val_loss: 0.5109 - val_acc: 0.6703
Epoch 4/10
1453/1453 [==============================] - 55s - loss: 0.0155 - acc: 0.9732 - val_loss: 0.5050 - val_acc: 0.6566
Epoch 5/10
1453/1453 [==============================] - 55s - loss: 0.0151 - acc: 0.9738 - val_loss: 0.5281 - val_acc: 0.6374
Epoch 6/10
1453/1453 [==============================] - 55s - loss: 0.0156 - acc: 0.9794 - val_loss: 0.4674 - val_acc: 0.6511
Epoch 7/10
1453/1453 [==============================] - 55s - loss: 0.0163 - acc: 0.9759 - val_loss: 0.4828 - val_acc

* Model v2: Network's test score [loss, accuracy]: [9.1067240715026863, 0.435] - 4474s 
  * #2: Network's test score [loss, accuracy]: [9.1067240715026863, 0.435] - 5469s 
* Model v3: Network's test score [loss, accuracy]: [8.0187525367736825, 0.50249999999999995] - 2499s
* Model v4.1: Network's test score [loss, accuracy]: [9.1067240715026863, 0.435] - 2137s 
* Movel v4.2: Network's test score [loss, accuracy]: [8.0187525367736825, 0.50249999999999995] - 3108s 
* Model v5: Network's test score [loss, accuracy]: [6.9666682052612305, 0.19500000000000001] - 1682s 
* Model v6: Network's test score [loss, accuracy]: [1.7120025205612182, 0.23749999999999999] - 3020s 
* Model v7: Network's test score [loss, accuracy]: [7.9999716758728026, 0.14000000000000001] - 4610s 
* Model v8: Network's test score [loss, accuracy]: [1.6948303937911988, 0.19500000000000001] - 3313s 
* Model v6 w/ flatten: Network's test score [loss, accuracy]: [7.1107604598999021, 0.17249999999999999] - 3044s 
* Model v6 (Docker Cloud): Network's test score [loss, accuracy]: [11.153776299942534, 0.307974335472044] - 3597s 
* Model v3.1: Network's test score [loss, accuracy]: [1.5994336946608279, 0.22314049603196873] - 687s 
* Model v3.2: Network's test score [loss, accuracy]: [1.5697537031802502, 0.35989011021760792] - 772s
* cnn_model_jhamski: Network's test score [loss, accuracy]: [0.31684105933367551, 0.34615384648134423] - 636s
 * Many epochs 150x150: Network's test score [loss, accuracy]: [0.51708218340690315, 0.6428571428571429]

In [24]:
model.save_weights("dwdii-emo-150vjh-Cloud.hdf5", overwrite=True)

### References

* OpenCV/CV2: http://askubuntu.com/questions/447409/how-to-install-opencv-2-9-for-python
* Docker Commit: http://stackoverflow.com/questions/19585028/i-lose-my-data-when-the-container-exits
  * docker ps -l
  * docker commit <ContainerID> dittenhafer/dl